In [5]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

from sklearn.model_selection import cross_val_score

from optuna import Trial
import optuna
from optuna.samplers import TPESampler

In [34]:
sample_df = pd.read_csv('train_data_final.csv', sep=',')

# 시간 순서대로 데이터 반영하기 위해서 거래년월일 정렬 필요
sample_df = sample_df.sort_values('계약날짜인코딩')
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237987 entries, 64104 to 208092
Data columns (total 19 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   전용면적         237987 non-null  float64
 1   거래금액         237987 non-null  int64  
 2   층            237987 non-null  int64  
 3   건축년도         237987 non-null  int64  
 4   평당단가         237987 non-null  float64
 5   구            237987 non-null  object 
 6   동            237987 non-null  object 
 7   단지명브랜드       237987 non-null  object 
 8   계약날짜인코딩      237987 non-null  int64  
 9   구별 공원 갯수     237987 non-null  float64
 10  8학군          237987 non-null  int64  
 11  동별지하철역수      237987 non-null  int64  
 12  구별 교과학원 갯수   237987 non-null  int64  
 13  구별 대형마트 수    235898 non-null  float64
 14  구별 백화점 수     235898 non-null  float64
 15  구별 전체 마트 수   235898 non-null  float64
 16  구별 대학병원 병원수  237987 non-null  int64  
 17  구별 종합병원 병원수  237987 non-null  int64  
 18  단지명별 신축거래비율  237987 

In [69]:
enc = OneHotEncoder()
encfit=enc.fit(sample_df['동'].values.reshape(-1, 1))
# enc.fit(X)
encfit

OneHotEncoder()

In [57]:
# encfit.categories_[0]

In [60]:
input_=np.array('공릉동').reshape(-1, 1)
enctrans=enc.transform(input_).toarray()

In [62]:
category_dong=sample_df['동'].drop_duplicates()

In [64]:
category_gu=sample_df['구'].drop_duplicates()

In [65]:
category_aptbrand=sample_df['단지명브랜드'].drop_duplicates()

In [67]:
category_dong.to_csv('category_dong.csv',index=False, encoding='utf-8-sig')
category_gu.to_csv('category_gu.csv',index=False, encoding='utf-8-sig')
category_aptbrand.to_csv('category_aptbrand.csv',index=False, encoding='utf-8-sig')

In [75]:
sample_df[['구','동','단지명브랜드','건축년도','층']].to_csv('category_df.csv',index=False, encoding='utf-8-sig')

In [76]:

def encoding_and_scaling(features):
    # category_gu = pd.read_csv('category_gu.csv',encoding='utf-8')
    # category_dong = pd.read_csv('category_dong.csv',encoding='utf-8')
    # category_aptbrand = pd.read_csv('category_aptbrand.csv',encoding='utf-8')
    category_df = pd.read_csv('category_df.csv', encoding='utf-8')
    enc_arr = np.array([])

    # 입력값마다 for문 반복
    # for feature in features:
    for i in range(5):
        # if feature.dtype == object or feature.dtype == str:
                # # 컬럼의 유니크한 값을 리스트로 만들어둠
            col_ith_items = category_df.iloc[:, i].unique().tolist()
            onehot = OneHotEncoder()
            onehot.fit(np.array(col_ith_items).reshape(-1, 1))

             # input 값 대입
            input_ = np.array(features[i]).reshape(-1, 1)
            col_ith_enc_arr = onehot.transform(input_).toarray()
            enc_arr = np.append(enc_arr, col_ith_enc_arr)

    return enc_arr

encoding_and_scaling(['강남구', '개포동', '현대',2007,2])

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.